In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%load_ext tensorboard

C:\Users\Piyush\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss = "sparse_categorical_crossentropy" , 
              optimizer = tf.keras.optimizers.SGD(lr = 1e-3) , metrics= ["accuracy"])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 0.5571 - accuracy: 0.8493 - val_loss: 0.3044 - val_accuracy: 0.9162 - 5s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 3s - loss: 0.2971 - accuracy: 0.9150 - val_loss: 0.2538 - val_accuracy: 0.9262 - 3s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.2537 - accuracy: 0.9276 - val_loss: 0.2207 - val_accuracy: 0.9366 - 4s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 3s - loss: 0.2240 - accuracy: 0.9363 - val_loss: 0.2009 - val_accuracy: 0.9424 - 3s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 3s - loss: 0.2015 - accuracy: 0.9428 - val_loss: 0.1835 - val_accuracy: 0.9466 - 3s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 3s - loss: 0.1827 - accuracy: 0.9484 - val_loss: 0.1705 - val_accuracy: 0.9518 - 3s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 3s - loss: 0.1671 - accuracy: 0.9526 - val_loss: 0.1575 - val_accuracy: 0.9564 - 3s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 3s - loss: 0.1537 - accuracy: 0.9561 - val_loss: 0.1516 - val_accuracy: 0.9578 - 3s/epoch - 2ms/step
Epoch 9/

In [7]:
model.save('full_model.h5')

# Transfer Learning

In [9]:
pretrained_model = tf.keras.models.load_model('./full_model.h5')

In [10]:
pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [12]:
pretrained_model.layers

In [14]:
for layer in pretrained_model.layers:
    print(f"{layer.name} : {layer.trainable}")

flatten : True
dense : True
leaky_re_lu : True
dense_1 : True
leaky_re_lu_1 : True
dense_2 : True


In [16]:
for layer in pretrained_model.layers[:-1]:
    layer.trainable = False
    print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False


In [17]:
for layer in pretrained_model.layers:
    print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False
dense_2 : True


In [18]:
lower_pretrained_layers = pretrained_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(2 , activation="softmax")

)

In [19]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [21]:
np.unique(y_train)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [22]:
np.where(y_train%2== 0 , 1 , 0)

array([0, 0, 1, ..., 0, 1, 1])

In [24]:
y_train[0] , y_train[-1]

(7, 8)

In [37]:
def update_odd_even_label (labels):
    for idx , label in enumerate(labels):
        labels[idx] =  np.where(label %2== 0 , 1 , 0)
    return labels

In [40]:
alpha = [1,2,3,4,5,6,7,8]
alp = update_odd_even_label(alpha)
alp

[array(0),
 array(1),
 array(0),
 array(1),
 array(0),
 array(1),
 array(0),
 array(1)]

In [41]:
y_train_bin , y_test_bin , y_valid_bin  =update_odd_even_label([y_train , y_test , y_valid])

In [45]:
np.unique(y_train_bin)

array([0, 1])

In [47]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [50]:
history = model.fit(X_train , y_train_bin ,epochs = 10, validation_data = (X_valid , y_valid_bin) , verbose = 1)

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1060 - accuracy: 0.9634 - val_loss: 0.0796 - val_accuracy: 0.9728
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.0843 - accuracy: 0.9705 - val_loss: 0.0845 - val_accuracy: 0.9688
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.0732 - accuracy: 0.9741 - val_loss: 0.0619 - val_accuracy: 0.9780
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.0661 - accuracy: 0.9777 - val_loss: 0.0604 - val_accuracy: 0.9786
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.0596 - accuracy: 0.9801 - val_loss: 0.0623 - val_accuracy: 0.9776
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.0546 - accuracy: 0.9817 - val_loss: 0.0707 - val_accuracy: 0.9726
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.0507 - accuracy: 0.9827 - val_loss: 0.0488 - val_accuracy:

In [51]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 1ms/step - loss: 0.8874 - accuracy: 0.5766


[0.8874170780181885, 0.5766000151634216]

In [52]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [59]:
np.argmax(new_model.predict(X_new), axis = -1)

1/1 [==============================] - 0s 17ms/step


array([0, 0, 0], dtype=int64)